# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [60]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import random

import thinkstats2
import thinkplot

## Multiple regression

Let's load up the NSFG data again.

In [61]:
import first

live, firsts, others = first.MakeFrames()

Here's birth weight as a function of mother's age (which we saw in the previous chapter).

In [62]:
import statsmodels.formula.api as smf

formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           5.72e-11
Time:                        08:30:46   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can extract the parameters.

In [63]:
inter = results.params['Intercept']
slope = results.params['agepreg']
inter, slope

(6.8303969733110375, 0.01745385147180325)

And the p-value of the slope estimate.

In [64]:
slope_pvalue = results.pvalues['agepreg']
slope_pvalue

5.722947107306718e-11

And the coefficient of determination.

In [65]:
results.rsquared

0.004738115474710591

The difference in birth weight between first babies and others.

In [66]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

The difference in age between mothers of first babies and others.

In [67]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

The age difference plausibly explains about half of the difference in weight.

In [68]:
slope * diff_age

-0.06259709972169616

Running a single regression with a categorical variable, `isfirst`:

In [69]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           2.55e-05
Time:                        08:30:46   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    356.007      0.000       7.286       7.366
isfirst[T.True]    -0.1248      0.030     -4.212      0.000      -0.183      -0.067
==============================================================================
Omnibus:                      988.919   Durbin-Watson:                   1.613
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2897.107
Skew:                          -0.589   Prob(JB):                         0.00
Kurtosis:                       5.511   Cond. No.                         2.58
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now finally running a multiple regression:

In [70]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           3.95e-11
Time:                        08:30:46   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As expected, when we control for mother's age, the apparent difference due to `isfirst` is cut in half.

If we add age squared, we can control for a quadratic relationship between age and weight.

In [71]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           1.35e-14
Time:                        08:30:46   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000       5.133       6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109      -0.112       0.011
agepreg             0.1124      0.022      5.113      0.000       0.069       0.155
agepreg2           -0.0018      0.000     -4.447      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

When we do that, the apparent effect of `isfirst` gets even smaller, and is no longer statistically significant.

These results suggest that the apparent difference in weight between first babies and others might be explained by difference in mothers' ages, at least in part.

## Data Mining

We can use `join` to combine variables from the preganancy and respondent tables.

In [72]:
import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

And we can search for variables with explanatory power.

Because we don't clean most of the variables, we are probably missing some good ones.

In [73]:
# Mine, just generalizing the function from the book to work with any dataset
# Did not include the agepreg variable though in mine

# import patsy

# def GoMining(df, y):
#     """Searches for variables that predict birth weight.

#     df: DataFrame of pregnancy records

#     returns: list of (rsquared, variable name) pairs
#     """
#     variables = []
#     for name in df.columns:
#         try:
#             if df[name].var() < 1e-7:
#                 continue

#             formula = '{} ~ '.format(y) + name
            
#             # The following seems to be required in some environments
#             # formula = formula.encode('ascii')

#             model = smf.ols(formula, data=df)
#             if model.nobs < len(df)/2:
#                 continue

#             results = model.fit()
#         except (ValueError, TypeError):
#             continue

#         variables.append((results.rsquared, name))

#     return variables

In [74]:
import patsy

def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'totalwgt_lb ~ agepreg + ' + name
            
            # The following seems to be required in some environments
            # formula = formula.encode('ascii')

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue

        variables.append((results.rsquared, name))

    return variables

In [75]:
variables = GoMining(join)

The following functions report the variables with the highest values of $R^2$.

In [76]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

Some of the variables that do well are not useful for prediction because they are not known ahead of time.

In [77]:
MiningReport(variables)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.30082407844707704 LOW BIRTHWEIGHT - BABY 1
prglngth 0.1301251948862503 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.12340041363361054 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.10203149928156086 AGE AT TIME OF CONCEPTION
mosgest 0.027144274639579802 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
babysex 0.01855092529394209 BD-2 SEX OF 1ST LIVEBORN BABY FROM THIS PREGNANCY
race_r 0.016199503586253217
race 0.016199503586253217
nbrnaliv 0.016017752709788224 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
paydu 0.014003795578114375 IB-10 CURRENT LIVING QUARTERS OWNED/RENTED, ETC
rmarout03 0.013430066465713542 INFORMAL MARITAL STATUS WHEN PREGNANCY ENDED - 3RD
birthwgt_oz 0.013102457615706053 BD-3 BIRTHWEIGHT IN OUNCES - 1ST BABY FROM THIS PREGNANCY
anynurse 0.012529022541810764 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bf

Combining the variables that seem to have the most explanatory power.

In [78]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Sun, 25 Oct 2020   Prob (F-statistic):          4.86e-113
Time:                        08:32:17   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.6303      0.065    102.223      0.000       6.503       6.757
C(race)[T.2]             0.3570      0.032     11.215      0.000       0.295       0.419
C(race)[T.3]             0.2665      0.051      5.175      0.000       0.166       0.367
babysex == 1[T.True]     0.2952      0.026     11.216      0.000       0.244       0.347
nbrnaliv > 1[T.True]    -1.3783      0.108    -12.771      0.000      -1.590      -1.167
paydu == 1[T.True]       0.1196      0.031      3.861      0.000       0.059       0.180
agepreg                  0.0074      0.003      2.921      0.004       0.002       0.012
totincr                  0.0122      0.004      3.110      0.002       0.005       0.020
==============================================================================
Omnibus:                      398.813   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1388.362
Skew:                          -0.037   Prob(JB):                    3.32e-302
Kurtosis:                       4.947   Cond. No.                         221.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Logistic regression

Example: suppose we are trying to predict `y` using explanatory variables `x1` and `x2`.

In [79]:
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

According to the logit model the log odds for the $i$th element of $y$ is

$\log o = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $

So let's start with an arbitrary guess about the elements of $\beta$:



In [80]:
beta = [-1.5, 2.8, 1.1]

Plugging in the model, we get log odds.

In [81]:
log_o = beta[0] + beta[1] * x1 + beta[2] * x2
log_o

array([-1.5, -0.4, -0.4,  2.4])

Which we can convert to odds.

In [82]:
o = np.exp(log_o)
o

array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638])

And then convert to probabilities.

In [83]:
p = o / (o+1)
p

array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ])

The likelihoods of the actual outcomes are $p$ where $y$ is 1 and $1-p$ where $y$ is 0. 

In [84]:
likes = np.where(y, p, 1-p)
likes

array([0.81757448, 0.40131234, 0.59868766, 0.9168273 ])

The likelihood of $y$ given $\beta$ is the product of `likes`:

In [85]:
like = np.prod(likes)
like

0.1800933529673034

Logistic regression works by searching for the values in $\beta$ that maximize `like`.

Here's an example using variables in the NSFG respondent file to predict whether a baby will be a boy or a girl.

In [86]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live['boy'] = (live.babysex==1).astype(int)

The mother's age seems to have a small effect.

In [87]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Sun, 25 Oct 2020   Pseudo R-squ.:               6.144e-06
Time:                        08:32:21   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                    0.7833
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953      -0.185       0.197
agepreg        0.0010      0.004      0.275      0.783      -0.006       0.009
==============================================================================
"""

Here are the variables that seemed most promising.

In [88]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Sun, 25 Oct 2020   Pseudo R-squ.:               0.0001440
Time:                        08:32:21   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
Covariance Type:            nonrobust   LLR p-value:                    0.8822
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772      -0.234       0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660      -0.122       0.077
C(race)[T.3]    -0.0005      0.083     -0.005      0.996      -0.163       0.162
agepreg         -0.0027      0.006     -0.484      0.629      -0.014       0.008
hpagelb          0.0047      0.004      1.112      0.266      -0.004       0.013
birthord         0.0050      0.022      0.227      0.821      -0.038       0.048
================================================================================
"""

To make a prediction, we have to extract the exogenous and endogenous variables.

In [89]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

The baseline prediction strategy is to guess "boy".  In that case, we're right almost 51% of the time.

In [90]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.507173764518333

If we use the previous model, we can compute the number of predictions we get right.

In [91]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3944.0, 548.0)

And the accuracy, which is slightly higher than the baseline.

In [92]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.5115007970849464

To make a prediction for an individual, we have to get their information into a `DataFrame`.

In [93]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

0    0.513091
dtype: float64

This person has a 51% chance of having a boy (according to the model).

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [94]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

The following are the only variables I found that have a statistically significant effect on pregnancy length.

In [95]:
import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Sun, 25 Oct 2020   Prob (F-statistic):           5.09e-22
Time:                        08:32:25   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [96]:
# Try out the author's summarize results function
def SummarizeResults(results):
    """Prints the most important parts of linear regression results:

    results: RegressionResults object
    """
    for name, param in results.params.items():
        pvalue = results.pvalues[name]
        print('%s   %0.3g   (%.3g)' % (name, param, pvalue))

    try:
        print('R^2 %.4g' % results.rsquared)
        ys = results.model.endog
        print('Std(ys) %.4g' % ys.std())
        print('Std(res) %.4g' % results.resid.std())
    except AttributeError:
        print('R^2 %.4g' % results.prsquared)

In [97]:
SummarizeResults(results)

Intercept   38.8   (0)
birthord == 1[T.True]   0.102   (0.0115)
race == 2[T.True]   0.139   (0.000935)
nbrnaliv > 1[T.True]   -1.49   (1.25e-19)
R^2 0.01145
Std(ys) 1.898
Std(res) 1.887


**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?

In [98]:
import regression
join = regression.JoinFemResp(live)

In [99]:
# Solution goes here
join['boy'] = (join.babysex==1).astype(int)

In [100]:
# Solution goes here
def VariableMiningLogit(df, y):
    """Searches variables using logistic regression to find ones that predict the target dependent variable 'y'.

    Args:
        df (DataFrame): DataFrame that holds all the variables.
        y (string): Column name of dependent variable y.

    Returns:
        variables (list): A list of tuples each containing r-squared value and variable name
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = '{} ~ '.format(y) + name
            model = smf.logit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

In [101]:
# Solution goes here
boy_variables = VariableMiningLogit(join, 'boy')
boy_variables

Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692962
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692850
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693001
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692766
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692995
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693017
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
  

Optimization terminated successfully.
         Current function value: 0.692895
         Iterations 5
         Current function value: 0.692784
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.692643
         Iterations 6


C:\Users\pthus\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692840
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.692975
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692975
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692978
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692977
  

Optimization terminated successfully.
         Current function value: 0.693030
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693099
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693108
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693056
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693127
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693128
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692964
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692801
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693079
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692966
  

...


         Current function value: 0.692997
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693014
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692993
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692973
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692673
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692917
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692807
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693018
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692838
         Iterations 3
Optimization term

Optimization terminated successfully.
         Current function value: 0.692951
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692866
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692910
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692966
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692981
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693005
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693016
         Iterations 3
         Current function value: 0.692943
         Iterations: 35
Optimization ter

C:\Users\pthus\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)



         Current function value: 0.693002
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692795
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692693
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692460
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692822
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693005
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692993
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693012
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 4
Optimization term

[(3.441548227833824e-05, 'caseid'),
 (8.285012918951562e-05, 'pregordr'),
 (0.0002445502560157742, 'pregend1'),
 (1.526619192793177e-05, 'nbrnaliv'),
 (0.00016779489634133338, 'cmprgend'),
 (0.0003851506826467732, 'cmprgbeg'),
 (8.125743363729399e-07, 'gestasun_m'),
 (3.486951913123093e-05, 'gestasun_w'),
 (2.777796931974219e-05, 'wksgest'),
 (4.333048782412252e-05, 'mosgest'),
 (0.00063513255179104, 'bpa_bdscheck1'),
 (0.009248981696271863, 'birthwgt_lb'),
 (1.5153902006570519e-06, 'birthwgt_oz'),
 (0.00016779489634133338, 'cmbabdob'),
 (2.6294555421912946e-10, 'kidage'),
 (9.503703868063429e-05, 'hpagelb'),
 (1.5475833461953137e-05, 'matchfound'),
 (0.00010693209724033093, 'anynurse'),
 (2.4752333694699757e-05, 'frsteatd_n'),
 (1.1892232548160742e-05, 'frsteatd_p'),
 (0.0007600842068936631, 'frsteatd'),
 (1.2675183948651636e-05, 'cmlastlb'),
 (2.381390901362579e-06, 'cmfstprg'),
 (4.1477288692526315e-07, 'cmlstprg'),
 (0.0003478498934222918, 'cmintstr'),
 (0.00025455670191198987, 'cm

In [102]:
MiningReport(boy_variables)

totalwgt_lb 0.009672005414744111
birthwgt_lb 0.009248981696271863 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
constat3 0.0010867058511312422 3RD PRIORITY CODE FOR CURRENT CONTRACEPTIVE STATUS
lbw1 0.0010485954907972772 LOW BIRTHWEIGHT - BABY 1
nplaced 0.0010069185324941277 # OF R'S BIO CHILDREN SHE PLACED FOR ADOPTION (BASED ON BPA)
infever 0.0008069106229561251 EVER USED INFERTILITY SERVICES OF ANY KIND
frsteatd 0.0007600842068936631 AGE (IN MOS) WHEN 1ST SUPPLEMENTED - 1ST FROM THIS PREG
splstwk1 0.0007317461972905503 IF-1 H/P DOING WHAT LAST WEEK (EMPLOYMENT STATUS) 1ST MENTION
outcom02 0.0006987412942979221 OUTCOME OF PREGNANCY - 2ND
fmarout5 0.0006819962814970104 FORMAL MARITAL STATUS AT PREGNANCY OUTCOME
nummult34 0.0006567469968998818 NUMBER OF METHODS REPORTED IN (OCT 2001)
coh1dur 0.0006549930318874297 DURATION (IN MONTHS) OF R'S FIRST COHABITATION
brnout_r 0.0006410689441835871
brnout 0.0006410689441835871
bpa_bdscheck1 0.00063513255179104 WHETHER 1ST LIVEBORN B

In [103]:
formula1 = 'boy ~ infever==1 + fmarout5==5'
model1 = smf.logit(formula1, data=join)
results1 = model1.fit()
results1.summary()

Optimization terminated successfully.
         Current function value: 0.691951
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8881
Method:                           MLE   Df Model:                            2
Date:                Sun, 25 Oct 2020   Pseudo R-squ.:                0.001542
Time:                        08:34:12   Log-Likelihood:                -6147.3
converged:                       True   LL-Null:                       -6156.8
Covariance Type:            nonrobust   LLR p-value:                 7.555e-05
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.0468      0.029     -1.624      0.104      -0.103       0.010
infever == 1[T.True]      0.2281      0.065      3.530      0.000       0.101       0.355
fmarout5 == 5[T.True]     0.1333      0.044      3.006      0.003       0.046       0.220
=========================================================================================
"""

In [104]:
df1=pd.DataFrame({'infever' : [1], 'fmarout5' : [5]})
df1

,infever,fmarout5
0,1,5


In [105]:
results1.predict(df1)

0    0.578009
dtype: float64

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [106]:
# Solution goes here
def VariableMiningPoisson(df, y):
    """Searches variables using Poisson regression to find ones that predict the target dependent variable 'y'.

    Args:
        df (DataFrame): DataFrame that holds all the variables.
        y (string): Column name of dependent variable y.

    Returns:
        variables (list): A list of tuples each containing r-squared value and variable name
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = '{} ~ '.format(y) + name
            model = smf.poisson(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

In [107]:
# Solution goes here
numbabes_variables = VariableMiningPoisson(join, 'numbabes')
numbabes_variables

Optimization terminated successfully.
         Current function value: 1.740484
         Iterations 16
Optimization terminated successfully.
         Current function value: 1.670513
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.736913
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.711854
         Iterations 8
Optimization terminated successfully.
         Current function value: 1.740969
         Iterations 15
Optimization terminated successfully.
         Current function value: 1.726723
         Iterations 14
Optimization terminated successfully.
         Current function value: 1.739591
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.739430
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741314
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741182

Optimization terminated successfully.
         Current function value: 1.735773
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.739343
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.738024
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740845
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.738321
         Iterations 6
Optimization terminated successfully.
         Current function value: 1.741153
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.740643
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.739656
         Iterations 6
Optimization terminated successfully.
         Current function value: 1.739748
         Iterations 6
Optimization terminated successfully.
         Current function value: 1.740931
  

Optimization terminated successfully.
         Current function value: 1.697979
         Iterations 15
Optimization terminated successfully.
         Current function value: 1.700770
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.699682
         Iterations 15
Optimization terminated successfully.
         Current function value: 1.701768
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.699338
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.701137
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.666381
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.701774
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.700677
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.641212


Optimization terminated successfully.
         Current function value: 1.731483
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.730907
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736051
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.782437
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.787525
         Iterations 15
Optimization terminated successfully.
         Current function value: 1.736219
         Iterations 15
Optimization terminated successfully.
         Current function value: 1.786543
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.720672
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740391
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.739267


Optimization terminated successfully.
         Current function value: 1.736843
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736545
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736798
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736455
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736662
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736460
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736713
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.736440
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.736663
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.736451
  

...

Optimization terminated successfully.
         Current function value: 1.741112
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740382
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.683070
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741027
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740603
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740718
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740951
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740786
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.738251
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741156
  

C:\Users\pthus\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 1.738167
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.789708
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.738923
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.734426
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.789083
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.738909
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.734426
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.789083
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.740287
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.738162
  

Optimization terminated successfully.
         Current function value: 1.723824
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.726974
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.740247
         Iterations 5
Optimization terminated successfully.
         Current function value: 1.740675
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.739204
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741172
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.740159
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741176
         Iterations 3
Optimization terminated successfully.
         Current function value: 1.740316
         Iterations 4
Optimization terminated successfully.
         Current function value: 1.741183
  

[(0.00040397714241913185, 'caseid'),
 (0.04058983802078142, 'pregordr'),
 (0.002454850493615268, 'pregend1'),
 (0.01486651986101628, 'nbrnaliv'),
 (0.0001255638272552595, 'cmprgend'),
 (0.005328034492925515, 'cmprgbeg'),
 (0.0009172474543542863, 'gestasun_m'),
 (0.0010094103635818197, 'gestasun_w'),
 (1.2311438525314244e-05, 'wksgest'),
 (3.5254895902614436e-06, 'mosgest'),
 (6.991749047502438e-07, 'bpa_bdscheck1'),
 (1.7688372249580198e-05, 'babysex'),
 (0.00012926316271422156, 'birthwgt_lb'),
 (0.0007025075743909426, 'birthwgt_oz'),
 (0.0001255638272552595, 'cmbabdob'),
 (0.0017059507925196726, 'kidage'),
 (2.6480682621565776e-05, 'hpagelb'),
 (0.0007881822777966452, 'matchfound'),
 (0.0025333271655012535, 'anynurse'),
 (5.87064892254574e-07, 'frsteatd_n'),
 (0.0013998042159839574, 'frsteatd_p'),
 (7.292752446386164e-05, 'frsteatd'),
 (3.987236985025788e-08, 'cmlastlb'),
 (0.013881551731196318, 'cmfstprg'),
 (1.9313726460357117e-06, 'cmlstprg'),
 (7.784343260808235e-05, 'cmintstr'),


In [108]:
MiningReport(numbabes_variables, n=60)

lbpregs 0.1661578912087034 CAPI-BASED TOTAL # OF PREGS ENDING IN LIVE BIRTH
parity_r 0.14478571719203237
parity 0.14478571719203237
numbabes 0.14478571719203237 NUMBER OF BABIES BORN ALIVE TO R (PARITY)
compreg 0.09939387956738122 CAPI-BASED TOTAL NUMBER OF COMPLETED PREGNANCIES
pregnum_r 0.0973448175410937
pregnum 0.0973448175410937
numpregs 0.0973448175410937 BB-1 NUMBER OF PREGNANCIES IN LIFETIME (INCLUDING CURRENT)
birthord 0.06915226259294127 BIRTH ORDER
numkdhh 0.06905045880207605 NUMBER OF BIO/ADOPT/RELATED/LEGAL CHILDREN UNDER AGE 18 IN HOUSEHOLD
roscnt 0.055339925623927866 NUMBER OF HOUSEHOLD MEMBERS BASED ON HH ROSTER
numfmhh 0.05465799754913114 NUMBER OF FAMILY MEMBERS IN HOUSEHOLD
cebow 0.051325760983849555 NUMBER OF CHILDREN BORN OUT OF WEDLOCK
pregordr 0.04058983802078142 PREGNANCY ORDER (NUMBER)
rostscrn 0.03906135133938771 # OF HOUSEHOLD MEMBERS BASED ON SCREENER
datbaby1 0.03849727508447742 CM DATE OF 1ST LIVE BIRTH
datcon01 0.031036534768801305 CM DATE WHEN PREGNANCY 

In [109]:
formula2 = 'numbabes ~ age_r + C(race) + totincr + educat'
model2 = smf.poisson(formula2, data=join)
results2 = model2.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Sun, 25 Oct 2020   Pseudo R-squ.:                 0.03109
Time:                        08:36:37   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [110]:
df_numbabes = pd.DataFrame({'age_r':[35], 'race':[1], 'totincr':[14], 'educat':[16]})
df_numbabes

,age_r,race,totincr,educat
0,35,1,14,16


In [111]:
# Solution goes here
results2.predict(df_numbabes)

0    2.342182
dtype: float64

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [112]:
# Solution goes here
def VariableMiningMnlogit(df, y):
    """Searches variables using multinomial logistic regression to find ones that predict the target dependent variable 'y'.

    Args:
        df (DataFrame): DataFrame that holds all the variables.
        y (string): Column name of dependent variable y.

    Returns:
        variables (list): A list of tuples each containing r-squared value and variable name
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = '{} ~ '.format(y) + name
            model = smf.mnlogit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [113]:
# Solution goes here: running this took far too long, so cancelled it
# rmarital_variables = VariableMiningMnlogit(join, 'rmarital')
# rmarital_variables

In [114]:
formula3 = 'rmarital ~ age_r + C(race) + totincr + educat'
model3 = smf.mnlogit(formula3, data=join)
results3 = model3.fit()
results3.summary()

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Sun, 25 Oct 2020   Pseudo R-squ.:                  0.1655
Time:                        08:36:39   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [115]:
df_rmarital = pd.DataFrame({'age_r':[25], 'race':[2], 'totincr':[11], 'educat':[12]})
df_rmarital

,age_r,race,totincr,educat
0,25,2,11,12


In [116]:
results3.predict(df_rmarital)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931
